## At this point, we have a bonafide IBC connection between injective and osmosis that is ready to service communication between any two "ports" (or smart contracts). 

## In this tutorial, we will setup confio's ica contracts (https://github.com/confio/cw-ibc-demo), with the controller contract (port) on injective and the host contract (port) on osmosis.

## One important note: during the channel setup process, the controller dispatches a separate IBC packet (message) that needs to be relayed with an hour timeout.

In [1]:
#imports

import pandas as pd
import json
import os
import sys
import base64
import requests
import subprocess
import math
import hashlib
import bech32
import time

from dateutil.parser import parse
from datetime import datetime, timedelta
from ecdsa import SECP256k1, SigningKey
from ecdsa.util import sigencode_string_canonize
from bech32 import bech32_decode, bech32_encode, convertbits
from google.protobuf.timestamp_pb2 import Timestamp as googTimestamp

from terra_sdk.client.lcd import LCDClient
from terra_sdk.core.wasm import MsgStoreCode, MsgInstantiateContract, MsgExecuteContract
from terra_sdk.core.bank import MsgSend
from terra_sdk.core.fee import Fee
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.core.bech32 import get_bech
from terra_sdk.core import AccAddress, Coin, Coins
from terra_sdk.client.lcd.api.tx import CreateTxOptions, SignerOptions
from terra_sdk.client.localterra import LocalTerra
from terra_sdk.core.wasm.data import AccessConfig
from terra_sdk.client.lcd.api._base import BaseAsyncAPI, sync_bind

from terra_proto.cosmwasm.wasm.v1 import AccessType
from terra_proto.cosmos.tx.v1beta1 import Tx, TxBody, AuthInfo, SignDoc, SignerInfo, ModeInfo, ModeInfoSingle, BroadcastTxResponse
from terra_proto.cosmos.base.abci.v1beta1 import TxResponse
from terra_proto.cosmos.tx.signing.v1beta1 import SignMode
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse
from terra_proto.ibc.core.channel.v1 import MsgChannelOpenInit, Channel, State, Order, Counterparty, MsgChannelOpenTry, MsgChannelOpenAck, MsgChannelOpenConfirm, QueryUnreceivedPacketsRequest, QueryUnreceivedPacketsResponse, QueryPacketCommitmentRequest, QueryPacketCommitmentResponse, Packet, QueryNextSequenceReceiveRequest, QueryNextSequenceReceiveResponse, MsgRecvPacket, MsgTimeout, QueryUnreceivedAcksRequest, QueryUnreceivedAcksResponse, MsgAcknowledgement
from terra_proto.ibc.core.connection.v1 import MsgConnectionOpenInit, Counterparty as ConnectionCounterParty, Version, MsgConnectionOpenTry, MsgConnectionOpenAck, MsgConnectionOpenConfirm
from terra_proto.ibc.lightclients.tendermint.v1 import ClientState, ConsensusState, Fraction, Header
from terra_proto.ics23 import HashOp, LengthOp, LeafOp, InnerOp, ProofSpec, InnerSpec, CommitmentProof, ExistenceProof, NonExistenceProof, BatchProof, CompressedBatchProof, BatchEntry, CompressedBatchEntry, CompressedExistenceProof, CompressedNonExistenceProof
from terra_proto.ibc.core.commitment.v1 import MerkleRoot, MerklePrefix, MerkleProof
from terra_proto.tendermint.types import ValidatorSet, Validator, SignedHeader, Header as tendermintHeader, Commit, BlockId, PartSetHeader, CommitSig, BlockIdFlag
from terra_proto.tendermint.version import Consensus
from terra_proto.tendermint.crypto import PublicKey
from betterproto.lib.google.protobuf import Any
from betterproto import Timestamp

#misc helper functions
sys.path.append(os.path.join(os.path.dirname(__name__), '..', 'scripts'))

from helpers import proto_to_binary, timestamp_string_to_proto, stargate_msg, create_ibc_client, fetch_chain_objects, bech32_to_hexstring, hexstring_to_bytes, bech32_to_b64, b64_to_bytes, fabricate_update_client, fetch_proofs, deploy_local_wasm, init_contract, execute_msg, fetch_channel_proof

In [2]:
#setup lcd clients, rpc urls, wallets
(inj, inj_wallet, inj_rpc_url, inj_rpc_header) = fetch_chain_objects("injective-888")
(osmo, osmo_wallet, osmo_rpc_url, osmo_rpc_header) = fetch_chain_objects("osmo-test-4")

In [3]:
#terra_proto imports
from terra_proto.ibc.core.channel.v1 import MsgChannelOpenInit, Channel, State, Order, Counterparty, MsgChannelOpenTry, MsgChannelOpenAck, MsgChannelOpenConfirm, QueryUnreceivedPacketsRequest, QueryUnreceivedPacketsResponse, QueryPacketCommitmentRequest, QueryPacketCommitmentResponse, Packet, QueryNextSequenceReceiveRequest, QueryNextSequenceReceiveResponse, MsgRecvPacket, MsgTimeout, QueryUnreceivedAcksRequest, QueryUnreceivedAcksResponse, MsgAcknowledgement
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse

In [4]:
#load ibc client & connection information from previous notebooks
context = {}
# Open the file for reading
with open("context.json", "r") as f:
    # Load the dictionary from the file
    context = json.load(f)
    
client_id_on_inj = context["client_id_on_inj"]
client_id_on_osmo = context["client_id_on_osmo"]
connection_id_on_inj = context["connection_id_on_inj"]
connection_id_on_osmo = context["connection_id_on_osmo"]

print(context)

{'client_id_on_inj': '07-tendermint-87', 'client_id_on_osmo': '07-tendermint-4298', 'connection_id_on_inj': 'connection-69', 'connection_id_on_osmo': 'connection-3714'}


## Setting up the smart contracts is the normal standard process, but ibc-enabled contracts will also automatically generate its IBC port.

In [5]:
#load wasms & instantiate contracts

##from scratch code upload
#ica_controller_code_id = deploy_local_wasm("/repos/cw-ibc-demo/artifacts/simple_ica_controller.wasm", inj_wallet, inj)
#osmo_ica_host_code_id = deploy_local_wasm("/repos/cw-ibc-demo/artifacts/simple_ica_host.wasm", osmo_wallet, osmo)
#osmo_cw1_code_id = deploy_local_wasm("/repos/cw-plus/artifacts/cw1_whitelist.wasm", osmo_wallet, osmo)

#using already uploaded code ids
ica_controller_code_id = '536'
osmo_ica_host_code_id = '4692'
osmo_cw1_code_id = '4693'

#controller
init_msg = {
}
controller_result = init_contract(ica_controller_code_id, init_msg, inj_wallet, inj, "ica_controller")
controller_address = controller_result.logs[0].events_by_type["wasm"]["_contract_address"][0]

#host
init_msg = {
  "cw1_code_id": int(osmo_cw1_code_id),
}
host_result = init_contract(osmo_ica_host_code_id, init_msg, osmo_wallet, osmo, "ica_host")
host_address = host_result.logs[0].events_by_type["instantiate"]["_contract_address"][0]

#contracts automatically generate an ibc port
controller_port = inj.wasm.contract_info(controller_address)["ibc_port_id"]
host_port = osmo.wasm.contract_info(host_address)["ibc_port_id"]

print(f"""
ica_controller_code_id on inj: {ica_controller_code_id}
ica_host_code_id on osmo: {osmo_ica_host_code_id}
cw1_code_id on osmo: {osmo_cw1_code_id}

controller_address: {controller_address}
host_address: {host_address}

controller_port: {controller_port}
host_port: {host_port}
""")



ica_controller_code_id on inj: 536
ica_host_code_id on osmo: 4692
cw1_code_id on osmo: 4693

controller_address: inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55
host_address: osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p53gkgyt28sjth5asu9up3s

controller_port: wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55
host_port: wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p53gkgyt28sjth5asu9up3s



## IBC channel setup is a 4-step handshake between the two ports (the controller and host smart contracts in this case) - 1) init, 2) try, 3) ack, 4) confirm.

## Connecting two IBC ports through an IBC channel is conceptually and procedurally analogous to connecting two IBC clients through an IBC connection.

###########################################################################################################################

## IBC channel step 1 - MsgChannelOpenInit

## Note that the port (the controller contract) has a hook that executes custom logic https://github.com/confio/cw-ibc-demo/blob/ce3112721e3b1d7650d848823e3055b2c2328d13/contracts/simple-ica-controller/src/ibc.rs#L24 

In [6]:
#fabricate message & dispatch via stargate

msg = MsgChannelOpenInit(
  port_id=controller_port,
  channel=Channel(
    state=State.STATE_INIT,
    ordering=Order.ORDER_UNORDERED,
    counterparty=Counterparty(
      port_id=host_port,
    ),
    connection_hops=[
      connection_id_on_inj
    ],
    version="simple-ica-v2",
  ),
  signer=inj_wallet.key.acc_address
)

channel_open_init_on_inj = stargate_msg("/ibc.core.channel.v1.MsgChannelOpenInit", msg, inj_wallet, inj)
channel_open_init_on_inj_df = pd.DataFrame(channel_open_init_on_inj["tx_response"]["logs"][0]["events"][0]["attributes"])
channel_id_on_inj = channel_open_init_on_inj_df[channel_open_init_on_inj_df["key"]=="channel_id"]["value"].values[0]

print(channel_open_init_on_inj_df)

                       key                                              value
0                  port_id    wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55
1               channel_id                                         channel-42
2     counterparty_port_id  wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p...
3  counterparty_channel_id                                                   
4            connection_id                                      connection-69
5                  version                                      simple-ica-v2


## IBC channel step 2 - MsgChannelOpenTry

## Note that the port (the host contract) also has a hook that executes custom logic https://github.com/confio/cw-ibc-demo/blob/ce3112721e3b1d7650d848823e3055b2c2328d13/contracts/simple-ica-host/src/contract.rs#L71

In [7]:
#update the client_id on osmosis (ie, refresh osmosis's knowledge of inj's tendermint state)

time.sleep(10) #wait a few blocks
msg = fabricate_update_client(inj, inj_rpc_url, inj_rpc_header, osmo, osmo_wallet, client_id_on_osmo)
update_client_before_channel_try_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", msg, osmo_wallet, osmo)
header_height = Header.FromString(msg.header.value).signed_header.header.height

inj_client_trusted_height = header_height
inj_client_trusted_revision_number = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")["client_state"]["latest_height"]["revision_number"]

print(header_height)

{'remote_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e83c6850>, 'remote_rpc_url': 'https://k8s.testnet.tm.injective.network:443/', 'remote_rpc_header': {}, 'client_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e839ddf0>, 'client_wallet': <terra_sdk.client.lcd.wallet.Wallet object at 0x7f87b9dc3e50>, 'client_id': '07-tendermint-4298'}



timestamp: seconds: 1678826951
nanos: 842542200
 


source current tendermint:
{'block_id': {'hash': '9wBRNENsVo8KeiX0tJpzkOAtAfOqz7uJje35xHJl3Xs=', 'part_set_header': {'total': 1, 'hash': '+r4qNMcFQoQ9I3IrLMOcDeHmu4d/L4zHL7SQxIePJPc='}}, 'block': {'header': {'version': {'block': '11', 'app': '0'}, 'chain_id': 'injective-888', 'height': '9200199', 'time': '2023-03-14T20:49:11.842542200Z', 'last_block_id': {'hash': 'tED/az4Napzrwc0+3sjL03pyTe/YE9VM6DlmtO1UNFM=', 'part_set_header': {'total': 1, 'hash': 'MMLAaIAzEYOsxiTOY5uj7h4soi2erDSKZkwkUvtdtgQ='}}, 'last_commit_hash': 'uGHhTi+07NR4br23vGigZNr1ROxvJMWH5yi8veqZSuA=',

In [8]:
#fetch channel proof

params = {
    "path": '"/store/ibc/key"',
    "data": "0x" + bytes(f"channelEnds/ports/{controller_port}/channels/{channel_id_on_inj}", "ascii").hex(),
    "prove": "true",
    "height": int(inj_client_trusted_height) - 1,
}
resp = requests.get(f"{inj_rpc_url}/abci_query", headers=inj_rpc_header, params=params).json()
proofs = [CommitmentProof.FromString(b64_to_bytes(x["data"])) for x in resp["result"]["response"]["proofOps"]["ops"]]
channel_proof = MerkleProof(proofs=proofs)


In [9]:
#fabricate message & dispatch via stargate

msg = MsgChannelOpenTry(
  port_id=host_port,
  channel=Channel(
    state=State.STATE_TRYOPEN,
    ordering=Order.ORDER_UNORDERED,
    counterparty=Counterparty(
      port_id=controller_port,
      channel_id=channel_id_on_inj,
    ),
    connection_hops=[
      connection_id_on_osmo
    ],
    version="simple-ica-v2",
  ),
  counterparty_version="simple-ica-v2",
  proof_init=channel_proof.SerializeToString(),
  proof_height=Height(int(inj_client_trusted_revision_number), int(inj_client_trusted_height)),
  signer=osmo_wallet.key.acc_address,
)

channel_open_try_result = stargate_msg("/ibc.core.channel.v1.MsgChannelOpenTry", msg, osmo_wallet, osmo)
print(channel_open_try_result)
channel_open_try_on_osmo_df = pd.DataFrame(channel_open_try_result["tx_response"]["logs"][0]["events"][0]["attributes"])
channel_id_on_osmo = channel_open_try_on_osmo_df[channel_open_try_on_osmo_df["key"]=="channel_id"]["value"].values[0]

print(channel_open_try_on_osmo_df)

{'tx_response': {'height': '9559746', 'txhash': 'BDD6103AA8CC1A65473E840955E1AF292AB1A91AB867DF67D1DCED4BA6206EE4', 'codespace': '', 'code': 0, 'data': '0A390A262F6962632E636F72652E6368616E6E656C2E76312E4D73674368616E6E656C4F70656E547279120F0A0D73696D706C652D6963612D7632', 'raw_log': '[{"events":[{"type":"channel_open_try","attributes":[{"key":"port_id","value":"wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p53gkgyt28sjth5asu9up3s"},{"key":"channel_id","value":"channel-3037"},{"key":"counterparty_port_id","value":"wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55"},{"key":"counterparty_channel_id","value":"channel-42"},{"key":"connection_id","value":"connection-3714"},{"key":"version","value":"simple-ica-v2"}]},{"type":"message","attributes":[{"key":"action","value":"/ibc.core.channel.v1.MsgChannelOpenTry"},{"key":"module","value":"ibc_channel"}]}]}]', 'logs': [{'msg_index': 0, 'log': '', 'events': [{'type': 'channel_open_try', 'attributes': [{'key': 'port_id', 'value': 'wasm.osmo1dya5jdc

## IBC channel step 3 - MsgChannelOpenAck

## Note that the port (the controller contract) has a hook that dispatches a separate IBC packet (message) https://github.com/confio/cw-ibc-demo/blob/ce3112721e3b1d7650d848823e3055b2c2328d13/contracts/simple-ica-controller/src/ibc.rs#L41

In [11]:
#update client on inj (ie, refresh inj's knowledge of osmosis's tendermint state) & fetch channel proof

time.sleep(10) #wait a few blocks
update_client_msg = fabricate_update_client(osmo, osmo_rpc_url, inj_rpc_header, inj, inj_wallet, client_id_on_inj)
update_client_before_channel_try_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", update_client_msg, inj_wallet, inj)
header_height = Header.FromString(update_client_msg.header.value).signed_header.header.height

osmo_client_trusted_height = header_height
osmo_client_trusted_revision_number = inj.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_inj}")["client_state"]["latest_height"]["revision_number"]

channel_proof = fetch_channel_proof(osmo_rpc_url, inj_rpc_header, host_port, channel_id_on_osmo, osmo_client_trusted_height, osmo_client_trusted_revision_number)

{'remote_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e839ddf0>, 'remote_rpc_url': 'https://rpc-test.osmosis.zone', 'remote_rpc_header': {}, 'client_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e83c6850>, 'client_wallet': <terra_sdk.client.lcd.wallet.Wallet object at 0x7f87e839dee0>, 'client_id': '07-tendermint-87'}



timestamp: seconds: 1678827024
nanos: 851533941
 


source current tendermint:
{'block_id': {'hash': 'Ajako6avHE19Iz24Y8spqegqwmOgsn3/foUclNrnb7c=', 'part_set_header': {'total': 1, 'hash': 'd/v24m/VzAkB6IXOK6QXQ0OxM1tBslNWYq+7Z/tYuO0='}}, 'block': {'header': {'version': {'block': '11', 'app': '16'}, 'chain_id': 'osmo-test-4', 'height': '9559755', 'time': '2023-03-14T20:50:24.851533941Z', 'last_block_id': {'hash': 'H0hKwrEcfAA0h76jl0XUiIqLXnuyysArjEuI7MyjeQc=', 'part_set_header': {'total': 1, 'hash': 'wnGOfuHNXJcvvA2959uXzAEl9YCn+ds5QD5okZHl4is='}}, 'last_commit_hash': 'XYT82c3Khzu98PCwj4LZVm55go6hr0GsHPobAcC+IsI=', 'data_hash': '47DE

In [13]:
#fabricate message & dispatch via stargate

msg = MsgChannelOpenAck(
  port_id=controller_port,
  channel_id=channel_id_on_inj,
  counterparty_channel_id=channel_id_on_osmo,
  counterparty_version="simple-ica-v2",
  proof_try=channel_proof.SerializeToString(),
  proof_height=Height(int(osmo_client_trusted_revision_number), int(osmo_client_trusted_height)),
  signer=inj_wallet.key.acc_address,
)

time.sleep(2)
channel_open_ack_result = stargate_msg("/ibc.core.channel.v1.MsgChannelOpenAck", msg, inj_wallet, inj)
channel_open_ack_on_inj_df = pd.DataFrame(channel_open_ack_result["tx_response"]["logs"][0]["events"][0]["attributes"])

print(channel_open_ack_on_inj_df)

                       key                                              value
0                  port_id    wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55
1               channel_id                                         channel-42
2     counterparty_port_id  wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p...
3  counterparty_channel_id                                       channel-3037
4            connection_id                                      connection-69


In [14]:
#parse the ibc who_am_i packet for relay after channel setup is complete
#the logic here is very specific to the ICA controller contract

packet_df = pd.DataFrame([x for x in channel_open_ack_result["tx_response"]["logs"][0]["events"] if x["type"]=="send_packet"][0]["attributes"])
packet_df["index"] = 0
packet_df = packet_df.pivot(index="index", columns="key", values="value")
packet_row = packet_df.iloc[0,:]

packet_to_relay = Packet(
  sequence=int(packet_row["packet_sequence"]),
  source_port=packet_row["packet_src_port"],
  source_channel=packet_row["packet_src_channel"],
  destination_port=packet_row["packet_dst_port"],
  destination_channel=packet_row["packet_dst_channel"],
  data=hexstring_to_bytes(packet_row["packet_data_hex"]),
  timeout_height=Height(int(packet_row["packet_timeout_height"].split('-')[0]), int(packet_row["packet_timeout_height"].split('-')[1])),
  timeout_timestamp=int(packet_row["packet_timeout_timestamp"]),
)

print(packet_to_relay)

Packet(sequence=1, source_port='wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55', source_channel='channel-42', destination_port='wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p53gkgyt28sjth5asu9up3s', destination_channel='channel-3037', data=b'{"who_am_i":{}}', timeout_height=Height(revision_number=0, revision_height=0), timeout_timestamp=1678830654441806271)


## IBC channel step 4 - MsgChannelOpenConfirm

## Note that the port (the host contract) hook https://github.com/confio/cw-ibc-demo/blob/ce3112721e3b1d7650d848823e3055b2c2328d13/contracts/simple-ica-host/src/contract.rs#L93

In [15]:
#update client again on osmosis & fetch proofs

time.sleep(10) #wait a few blocks
update_client_msg = fabricate_update_client(inj, inj_rpc_url, inj_rpc_header, osmo, osmo_wallet, client_id_on_osmo)
update_client_before_channel_try_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", update_client_msg, osmo_wallet, osmo)
header_height = Header.FromString(update_client_msg.header.value).signed_header.header.height

inj_client_trusted_height = header_height
inj_client_trusted_revision_number = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")["client_state"]["latest_height"]["revision_number"]

channel_proof = fetch_channel_proof(inj_rpc_url, inj_rpc_header, controller_port, channel_id_on_inj, inj_client_trusted_height, inj_client_trusted_revision_number)

{'remote_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e83c6850>, 'remote_rpc_url': 'https://k8s.testnet.tm.injective.network:443/', 'remote_rpc_header': {}, 'client_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e839ddf0>, 'client_wallet': <terra_sdk.client.lcd.wallet.Wallet object at 0x7f87b9dc3e50>, 'client_id': '07-tendermint-4298'}



timestamp: seconds: 1678827089
nanos: 63203456
 


source current tendermint:
{'block_id': {'hash': 'GkA0sPMn72HXWZ4zVTA5XuDdEdYRSBEnIYFUtKHarII=', 'part_set_header': {'total': 1, 'hash': 'af9RbxpZrXkJzUGwUfdP5JTTnYXOHj67jOCARcXSeEE='}}, 'block': {'header': {'version': {'block': '11', 'app': '0'}, 'chain_id': 'injective-888', 'height': '9200255', 'time': '2023-03-14T20:51:29.063203456Z', 'last_block_id': {'hash': 'equH0+OglqaYDLpyYWdOHE72IpPk6k8eA3ckGdEg9+I=', 'part_set_header': {'total': 1, 'hash': 'Ii2ZZlSDlxJF0nhT3Sz2rua2DFG33Y/LHrXxqelQ6B8='}}, 'last_commit_hash': 'D72qCquefDup4AwBYXEbguI4FQUgsAAvGZQuvdnxizY=', 

In [16]:
#fabricate message & dispatch via stargate

msg = MsgChannelOpenConfirm(
  port_id=host_port,
  channel_id=channel_id_on_osmo,
  proof_ack=channel_proof.SerializeToString(),
  proof_height=Height(int(inj_client_trusted_revision_number), int(inj_client_trusted_height)),
  signer=osmo_wallet.key.acc_address,
)

channel_open_confirm_result = stargate_msg("/ibc.core.channel.v1.MsgChannelOpenConfirm", msg, osmo_wallet, osmo)
channel_open_confirm_on_osmo_df = pd.DataFrame(channel_open_confirm_result["tx_response"]["logs"][0]["events"][0]["attributes"])

print(channel_open_confirm_on_osmo_df)

                       key                                              value
0                  port_id  wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p...
1               channel_id                                       channel-3037
2     counterparty_port_id    wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55
3  counterparty_channel_id                                         channel-42
4            connection_id                                    connection-3714


## At this point, the IBC channel connecting the ica contracts is setup! 

## However, the IBC packet dispatched in the ChannelOpenAck step needs to be relayed/acked to complete the ica contract setup (specific to the contracts).

In [17]:
#update client again on osmosis & fetch the packet proof

time.sleep(10) #wait a few blocks
update_client_msg = fabricate_update_client(inj, inj_rpc_url, inj_rpc_header, osmo, osmo_wallet, client_id_on_osmo)
update_client_before_channel_try_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", update_client_msg, osmo_wallet, osmo)
header_height = Header.FromString(update_client_msg.header.value).signed_header.header.height

inj_client_trusted_height = header_height
inj_client_trusted_revision_number = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")["client_state"]["latest_height"]["revision_number"]

params = {
    "path": '"/store/ibc/key"',
    "data": "0x" + bytes(f"commitments/ports/{controller_port}/channels/{channel_id_on_inj}/sequences/{packet_to_relay.sequence}", "ascii").hex(),
    "prove": "true",
    "height": int(inj_client_trusted_height) - 1,
}
resp = requests.get(f"{inj_rpc_url}/abci_query", headers=inj_rpc_header, params=params).json()
proofs = [CommitmentProof.FromString(b64_to_bytes(x["data"])) for x in resp["result"]["response"]["proofOps"]["ops"]]
packet_proof = MerkleProof(proofs=proofs)

{'remote_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e83c6850>, 'remote_rpc_url': 'https://k8s.testnet.tm.injective.network:443/', 'remote_rpc_header': {}, 'client_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e839ddf0>, 'client_wallet': <terra_sdk.client.lcd.wallet.Wallet object at 0x7f87b9dc3e50>, 'client_id': '07-tendermint-4298'}



timestamp: seconds: 1678827123
nanos: 63654166
 


source current tendermint:
{'block_id': {'hash': 'eJ9yQdBdwvz5oahvv+HQ5ZUEY3avEe76nIOr4oJa2+s=', 'part_set_header': {'total': 1, 'hash': 'yhfAsi6EoZ9YAv6SinLvvOTerIGchfbnhZNhzxLnv5Y='}}, 'block': {'header': {'version': {'block': '11', 'app': '0'}, 'chain_id': 'injective-888', 'height': '9200269', 'time': '2023-03-14T20:52:03.063654166Z', 'last_block_id': {'hash': 'ljSLeUSACsye0UhIw57KEWGMoePGHlLb6P58y2V7F9A=', 'part_set_header': {'total': 1, 'hash': 'NHsBkrDY8Lh8yBBQfw2V7XloF4UjBXHSUY5bA/F+KMo='}}, 'last_commit_hash': 'u//nEwTy7zgs2dZJXGCjRKIEYY7INvdWKK5+G40ULUM=', 

In [18]:
#fabricate message & dispatch via stargate
msg = MsgRecvPacket(
  packet=packet_to_relay,
  proof_commitment=packet_proof.SerializeToString(),
  proof_height=Height(int(inj_client_trusted_revision_number), int(inj_client_trusted_height)),
  signer=osmo_wallet.key.acc_address
)

relay_result = stargate_msg("/ibc.core.channel.v1.MsgRecvPacket", msg, osmo_wallet, osmo)
relay_result_df = pd.DataFrame([y for x in [x["attributes"] for x in relay_result["tx_response"]["logs"][0]["events"]] for y in x])

print(relay_result_df)

                         key  \
0                     action   
1                     module   
2                     module   
3                packet_data   
4            packet_data_hex   
5      packet_timeout_height   
6   packet_timeout_timestamp   
7            packet_sequence   
8            packet_src_port   
9         packet_src_channel   
10           packet_dst_port   
11        packet_dst_channel   
12   packet_channel_ordering   
13         packet_connection   
14         _contract_address   
15                    action   
16               packet_data   
17           packet_data_hex   
18     packet_timeout_height   
19  packet_timeout_timestamp   
20           packet_sequence   
21           packet_src_port   
22        packet_src_channel   
23           packet_dst_port   
24        packet_dst_channel   
25                packet_ack   
26            packet_ack_hex   
27         packet_connection   

                                                value  
0              

In [19]:
#parse the acknowledge packet (message) for the ack relay back to the controller smart contract

ack_df = pd.DataFrame([x for x in relay_result["tx_response"]["logs"][0]["events"] if x["type"]=="write_acknowledgement"][0]["attributes"])
ack_df["index"] = 0
ack_df = ack_df.pivot(index="index", columns="key", values="value")
ack_row = ack_df.iloc[0,:]

print(ack_row)

key
packet_ack                  {"result":"eyJhY2NvdW50Ijoib3NtbzFhMnhmZHFmODV...
packet_ack_hex              7b22726573756c74223a2265794a6859324e7664573530...
packet_connection                                             connection-3714
packet_data                                                   {"who_am_i":{}}
packet_data_hex                                7b2277686f5f616d5f69223a7b7d7d
packet_dst_channel                                               channel-3037
packet_dst_port             wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p...
packet_sequence                                                             1
packet_src_channel                                                 channel-42
packet_src_port               wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55
packet_timeout_height                                                     0-0
packet_timeout_timestamp                                  1678830654441806271
Name: 0, dtype: object


## The packet acknowledgement contains the host contract's response to the controller contract's who_am_i request. The final relaying of this ack concludes successful setup of an ibc channel between two ica contract ports.

In [21]:
#update client on inj & fetch the acknowledge proof

time.sleep(10) #wait a few blocks
update_client_msg = fabricate_update_client(osmo, osmo_rpc_url, inj_rpc_header, inj, inj_wallet, client_id_on_inj)
update_client_before_channel_try_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", update_client_msg, inj_wallet, inj)
header_height = Header.FromString(update_client_msg.header.value).signed_header.header.height

osmo_client_trusted_height = header_height
osmo_client_trusted_revision_number = inj.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_inj}")["client_state"]["latest_height"]["revision_number"]

##fetch packetack proofs - acks/ports/{port_id}/channels/{channel_id}/sequences/{ack_sequence}
params = {
  "path": '"/store/ibc/key"',
  "data": "0x" + bytes(f"acks/ports/{ack_row['packet_dst_port']}/channels/{ack_row['packet_dst_channel']}/sequences/{ack_row['packet_sequence']}", "ascii").hex(),
  "prove": "true",
  "height": int(osmo_client_trusted_height) - 1,
}
resp = requests.get(f"{osmo_rpc_url }/abci_query", headers=inj_rpc_header, params=params).json()
proofs = [CommitmentProof.FromString(b64_to_bytes(x["data"])) for x in resp["result"]["response"]["proofOps"]["ops"]]
ack_proof = MerkleProof(proofs=proofs)

print(ack_proof)

{'remote_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e839ddf0>, 'remote_rpc_url': 'https://rpc-test.osmosis.zone', 'remote_rpc_header': {}, 'client_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f87e83c6850>, 'client_wallet': <terra_sdk.client.lcd.wallet.Wallet object at 0x7f87e839dee0>, 'client_id': '07-tendermint-87'}



timestamp: seconds: 1678827190
nanos: 415301036
 


source current tendermint:
{'block_id': {'hash': '1WoqINFc92Ta/OJwzFnPChsVZC7NNVUPdjil8J9xePc=', 'part_set_header': {'total': 1, 'hash': 'gC0KbxrWTAuKqGIhwNoSrsvTSlt5G/YYjQuVIl5nt7s='}}, 'block': {'header': {'version': {'block': '11', 'app': '16'}, 'chain_id': 'osmo-test-4', 'height': '9559788', 'time': '2023-03-14T20:53:10.415301036Z', 'last_block_id': {'hash': '9WIKfZv518KaNYa8l4oOwKKBg4/BRbytXzm+6gYztsU=', 'part_set_header': {'total': 1, 'hash': 'TaWnqEPzWbD29scOETMMv9ear5eTAT7YC7cDpOOGZmQ='}}, 'last_commit_hash': 'ExDsoAMLM1RcDd+CXnrCnaoej4WzeUi8PFUp34KOZxI=', 'data_hash': 'mX6I

In [23]:
#fabricate message & dispatch via stargate

msg = MsgAcknowledgement(
  packet=packet_to_relay,
  acknowledgement=hexstring_to_bytes(ack_row["packet_ack_hex"]),
  proof_acked=ack_proof.SerializeToString(),
  proof_height=Height(int(osmo_client_trusted_revision_number), int(osmo_client_trusted_height)),
  signer=inj_wallet.key.acc_address,
)

relay_ack_result = stargate_msg("/ibc.core.channel.v1.MsgAcknowledgement", msg, inj_wallet, inj)
print(relay_ack_result)
relay_ack_result_df = pd.DataFrame([y for x in [x["attributes"] for x in relay_ack_result["tx_response"]["logs"][0]["events"]] for y in x])

print(relay_ack_result_df)

{'tx_response': {'height': '9200304', 'txhash': 'C8428AD4A5B7D5C0FBD360DEC1AC81E65B69678B3EC146272CDBBAA863EC1B79', 'codespace': '', 'code': 0, 'data': '0A2D0A272F6962632E636F72652E6368616E6E656C2E76312E4D736741636B6E6F776C656467656D656E7412020802', 'raw_log': '[{"events":[{"type":"acknowledge_packet","attributes":[{"key":"packet_timeout_height","value":"0-0"},{"key":"packet_timeout_timestamp","value":"1678830654441806271"},{"key":"packet_sequence","value":"1"},{"key":"packet_src_port","value":"wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55"},{"key":"packet_src_channel","value":"channel-42"},{"key":"packet_dst_port","value":"wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p53gkgyt28sjth5asu9up3s"},{"key":"packet_dst_channel","value":"channel-3037"},{"key":"packet_channel_ordering","value":"ORDER_UNORDERED"},{"key":"packet_connection","value":"connection-69"}]},{"type":"message","attributes":[{"key":"action","value":"/ibc.core.channel.v1.MsgAcknowledgement"},{"key":"module","value":"ibc_c

In [25]:
#query the controller contract to confirm its account state reflects
print(inj.wasm.contract_query(controller_address, {"list_accounts":{}}))

{'accounts': [{'channel_id': 'channel-42', 'last_update_time': '0', 'remote_addr': 'osmo1a2xfdqf85uzf2zv5c3kr4x2wz4p3ya8kqtjkeg8jk5s4ylfmdwnsf59892', 'remote_balance': []}]}


In [26]:
#persist channel and port information for regular packet/ack relaying in the next notebook (optional)

context["channel_id_on_inj"] = channel_id_on_inj
context["channel_id_on_osmo"] = channel_id_on_osmo
context["port_id_on_inj"] = controller_port
context["port_id_on_osmo"] = host_port
context["controller_address"] = controller_address
context["host_address"] = host_address
context["last_relayed_height_on_inj"] = inj_client_trusted_height
context["last_relayed_height_on_osmo"] = osmo_client_trusted_height
context["last_ack_height_on_inj"] = inj_client_trusted_height
context["last_ack_height_on_osmo"] = osmo_client_trusted_height

print(context)

with open("context.json", "w") as f:
    # Write the dictionary to the file as a JSON string
    json.dump(context, f)

{'client_id_on_inj': '07-tendermint-87', 'client_id_on_osmo': '07-tendermint-4298', 'connection_id_on_inj': 'connection-69', 'connection_id_on_osmo': 'connection-3714', 'channel_id_on_inj': 'channel-42', 'channel_id_on_osmo': 'channel-3037', 'port_id_on_inj': 'wasm.inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55', 'port_id_on_osmo': 'wasm.osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p53gkgyt28sjth5asu9up3s', 'controller_address': 'inj10mp9hu2mjh7md53t2adp9s70cn8dfjsnnd4w55', 'host_address': 'osmo1dya5jdc5dqnqvf8y8j03x22ptrw2w7tkrz7p53gkgyt28sjth5asu9up3s', 'last_relayed_height_on_inj': 9200269, 'last_relayed_height_on_osmo': 9559788, 'last_ack_height_on_inj': 9200269, 'last_ack_height_on_osmo': 9559788}
